# Assessing the ICON-ModEx ML models

## How does the model score change with each ICON-ModEx iteration?
This notebook will harvest the data from the `dynamic-learning-rivers` repository to make a plot showing the evolution of the ML models as more traininig data is added.

## Important behind the scenes information
Here, I am assuming that this notebook is running in a Linux context (e.g. pull a [container that runs a local Jupyter server](https://hub.docker.com/r/jupyter/datascience-notebook) and open the notebook therein). This code will switch branches within a repository that it downloads from GitHub to load the data for side by side comparisions. It will then delete the source repository and retain only the harvested data used for plotting. The harvested data will be saved alongside this notebook here. 

In [1]:
# Run once at start of notebook if these 
# packages are not already installed
! conda install -y -q pandas numpy matplotlib
! conda install -y -q papermill

# Packages needed for R notebook
# Pulls binaries from conda-forge, much faster than
# installing direct from CRAN (and building from source).
# ALso, some packages have linking errors when building 
# from source.
! conda install -y -q -c conda-forge r-tidyverse
! conda install -y -q -c conda-forge r-colourpicker
! conda install -y -q -c conda-forge r-emmeans
# Present in r-ggspatial
#! conda install -y -q -c conda-forge r-sf
! conda install -y -q -c conda-forge r-ggspatial

# Convert from .Rmd. Not used
#! conda install -y -q jupytext

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/sfgary/pw/.miniconda3c

  added / updated specs:
    - matplotlib
    - numpy
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.3.7           |  py311hf4808d0_0         145 KB
    brotli-1.0.9               |       h5eee18b_8          18 KB
    brotli-bin-1.0.9           |       h5eee18b_8          19 KB
    contourpy-1.2.0            |  py311hdb19cb5_0         263 KB
    cycler-0.11.0              |     pyhd3eb1b0_0          12 KB
    cyrus-sasl-2.1.28          |       h52b45da_1         237 KB
    dbus-1.13.18               |       hb2f20db_0         504 KB
    fonttools-4.51.0           |  py311h5eee18b_0         3.0 MB
    gst-plugins-base-1.14.1    |       h6a678d5_1         2.2 MB
    gstreamer-1.14.1           

In [2]:
import pandas as pd
import numpy as np
import papermill as pm
import matplotlib.pyplot as plt

# Convert .Rmd subscript to .ipynb
#! jupytext --to notebook make_scatter_plot.Rmd

Matplotlib is building the font cache; this may take a moment.


## Clone source repository

In [3]:
repo_prefix = '~/tmp/'
repo_org='parallelworks'
repo_name='dynamic-learning-rivers'
repo_url = 'https://github.com/'+repo_org+'/'+repo_name

! mkdir -p {repo_prefix}
! cd {repo_prefix}; git clone {repo_url}

Cloning into 'dynamic-learning-rivers'...
remote: Enumerating objects: 15243, done.
remote: Counting objects: 100% (3066/3066), done.
remote: Compressing objects: 100% (1599/1599), done.
remote: Total 15243 (delta 1485), reused 2983 (delta 1430), pack-reused 12177
Receiving objects: 100% (15243/15243), 3.24 GiB | 49.66 MiB/s, done.
Resolving deltas: 100% (7783/7783), done.


## Grab data and save to CSV
Here we want to track the model score over ModEx iterations.

In [4]:
# Functionalize all the data preporatory steps based on the
# branch name.
def prep_data_on_branch(repo_prefix=repo_prefix, repo_name=repo_name, branch_name='main'):

    # Change to the requested branch
    ! cd {repo_prefix}/{repo_name}; git checkout {branch_name}
    
    # Create a working directory for this branch
    ! mkdir -p ./intermediate_branch_data/{branch_name}
    
    # Load data we want to prepare/merge/preprocess
    data_prefix=repo_prefix+'/'+repo_name
    main_input = pd.read_csv(data_prefix+"/scripts/prep_01_output_train.csv")
    avg_output = pd.read_csv(data_prefix+'/scripts/post_01_output_ml_predict_avg.csv')
    std_output = pd.read_csv(data_prefix+'/scripts/post_01_output_ml_predict_std.csv')
    
    # Use Sample_ID as the index so files can be merged on this value
    main_input.set_index('Sample_ID',inplace=True)
    avg_output.set_index('Sample_ID',inplace=True)
    std_output.set_index('Sample_ID',inplace=True)

    # Merge files
    avg_merge = pd.merge(
        main_input,
        avg_output, 
        left_index=True,
        right_index=True,
        suffixes=['_obs','_pre_avg'])
    
    # Add log calculations for use with R notebook
    avg_merge['Log_Observed_Normalized_Respiration_Rate'] = np.log10(np.abs(avg_merge['Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_obs']))
    avg_merge['Log_Predicted_Normalized_Respiration_Rate'] = np.log10(np.abs(avg_merge['Normalized_Respiration_Rate_mg_DO_per_H_per_L_sediment_pre_avg']))
    
    all_merge = pd.merge(
        avg_merge,
        std_output, 
        left_index=True,
        right_index=True,
        suffixes=['','_pre_std'])

    predict_and_obs_merge = pd.merge(
        avg_output,
        std_output,
        left_index=True,
        right_index=True,
        suffixes=['_pre_avg','_pre_std'])
    
    # Write intermediate output files
    # Name it the same as what the R notebook expects...
    avg_merge.to_csv('intermediate_branch_data/'+branch_name+'/ICON-ModEx_Combined_Predicted_Observed_Respiration_Rates.csv',mode='w')
    all_merge.to_csv('intermediate_branch_data/'+branch_name+'/output_obs_avgpre_stdpre_merged.csv',mode='w')
    predict_and_obs_merge.to_csv('intermediate_branch_data/'+branch_name+'/output_all_sites_avgpre_stdpre_merged.csv',mode='w')


In [5]:
# Start from fresh! Delete any .csv files that may already be here
!rm -f ICON-ModEx_automated_iterations.csv

# WORKING HERE! Move to inside the loop!
# Use Papermill to launch a notebook
# To list available kernels,
#   jupyter kernelspec list
# The working directory was created by prep_data_on_branch
# The first R kernel execution takes a few extra minutes as
# necessary R packages are installed on-the-fly during the 
# first execution of the notebook (but subsequent reruns with
# the same kernel do not need to reinstall).    
#pm.execute_notebook(
#    'make_scatter_plot.ipynb',
#    'log_scatter_plot.ipynb',
#    parameters=dict(work_dir='./intermediate_branch_data/Oct-2023-log10', hello='world'),
#    kernel_name='ir'
#)

# List the branches we want to work with
list_branches_scale = [
    'Jul-2022',
    'Aug-2022',
    'Sep-2022',
    'Oct-2022',
    'Nov-2022',
    'Dec-2022',
    'Jan-2023',
    'Feb-2023',
    'Mar-2023',
    'ICON-ModEx-v2.1-April-2023-replicate-01',
    'ICON-ModEx-v2.1-April-2023-replicate-02',
    'May-2023',
    'June-2023',
    'Jul-2023',
    'August-2023',
    'Sep-2023',
    'Oct-2023',
    'Nov-2023']

list_branches_log10 = [
    'Dec-2021-log10',
    'Sep-2019-log10',
    'Jul-2022-log10',
    'Aug-2022-log10',
    'Sep-2022-log10',
    'Oct-2022-log10',
    'Nov-2022-log10',
    'Dec-2022-log10',
    'Jan-2023-log10',
    'Feb-2023-log10',
    'Mar-2023-log10',
    'Apr-2023-log10',
    'May-2023-log10',
    'June-2023-log10',
    'Jul-2023-log10',
    'August-2023-log10',
    'Sep-2023-log10',
    'Oct-2023-log10',
    'Nov-2023-log10',
    'Nov-2023-log10-DO-update-correct']

# Initialize empty dataframes
icon_modex_log10 = pd.DataFrame(columns=['hold-out-avg','hold-out-std','ntrain'])
icon_modex_scale = pd.DataFrame(columns=['hold-out-avg','hold-out-std','ntrain'])

#=======================================
# Gather data over the standard branches
#=======================================
for bb,branch in enumerate(list_branches_scale):
    
    # Change to the requested branch
    # We do not need to prep_data_on_branch for the non-log10 branches
    # since we are not plotting the animation of the scatter 
    #prep_data_on_branch(repo_prefix=repo_prefix, repo_name=repo_name, branch_name=branch)
    !cd {repo_prefix}/{repo_name}; git checkout {branch}
    
    # We want to get the mean and std of the score and plot that vs the number of inputs to train.
    # Each dot will be an ICON-ModEx iteration.
    
    # This file summarizes the holdout score for all the
    # Superlearners for each ICON-ModEx iteration.
    holdout_score = pd.read_csv(
        repo_prefix+'/'+repo_name+'/output_data/holdout_score.txt', 
        sep=':', 
        names=['Value','Number'], 
        index_col='Value' )
    #holdout_score = pd.read_csv(repo+'/output_data/holdout_score.txt', sep=':')
    
    # This file lists the total number of data available
    # (of which 25% is split to the testing set)
    # Some of these points are dropped during training due to
    # missing values. A better measure of the actual number of
    # data points used to train the models is in the 
    # alternative file which is present after all the 
    # preprocessing steps are completed.
    # RAW INPUT
    #input_data = pd.read_csv(repo+'/input_data/ICON-ModEx_Data.csv')
    # PREPROCESSED INPUT
    input_data = pd.read_csv(repo_prefix+'/'+repo_name+'/scripts/prep_06_output_final_train.csv')
    
    icon_modex_scale.loc[len(icon_modex_scale.index)] = [
        holdout_score.loc['Avg',:].values[0],
        holdout_score.loc['Std',:].values[0],
        len(input_data.index)
    ]
    
    # We could also grab the histogram of CONUS predictions to 
    # check for an evolution of bias over time
    # Grab code from the Figure-2 harvestor notebook if needed.    

#=====================================
# Gather data over the -log10 branches
#=====================================
for bb,branch in enumerate(list_branches_log10):
    
    # Change to the requested branch
    prep_data_on_branch(repo_prefix=repo_prefix, repo_name=repo_name, branch_name=branch)
    #!cd {repo_prefix}/{repo_name}; git checkout {branch}
    
    # We want to get the mean and std of the score and plot that vs the number of inputs to train.
    # Each dot will be an ICON-ModEx iteration.
    
    # This file summarizes the holdout score for all the
    # Superlearners for each ICON-ModEx iteration.
    holdout_score = pd.read_csv(
        repo_prefix+'/'+repo_name+'/output_data/holdout_score.txt', 
        sep=':', 
        names=['Value','Number'], 
        index_col='Value' )
    #holdout_score = pd.read_csv(repo+'/output_data/holdout_score.txt', sep=':')
    
    # This file lists the total number of data available
    # (of which 25% is split to the testing set)
    # RAW INPUT (see note above)
    #input_data = pd.read_csv(repo+'/input_data/ICON-ModEx_Data.csv')
    # PREPROCESSED INPUT
    input_data = pd.read_csv(repo_prefix+'/'+repo_name+'/scripts/prep_06_output_final_train.csv')
    
    icon_modex_log10.loc[len(icon_modex_log10.index)] = [
        holdout_score.loc['Avg',:].values[0],
        holdout_score.loc['Std',:].values[0],
        len(input_data.index)
    ]
    
    # We could also grab the histogram of CONUS predictions to check for an evolution of bias over time
    # Grab code from the Figure-2 harvestor notebook if needed.

# Save the intermediate results for reproducibility and
# offline plotting


branch 'Jul-2022' set up to track 'origin/Jul-2022'.
Switched to a new branch 'Jul-2022'
branch 'Aug-2022' set up to track 'origin/Aug-2022'.
Switched to a new branch 'Aug-2022'
Updating files: 100% (185/185), done.
branch 'Sep-2022' set up to track 'origin/Sep-2022'.
Switched to a new branch 'Sep-2022'
Updating files: 100% (185/185), done.
branch 'Oct-2022' set up to track 'origin/Oct-2022'.
Switched to a new branch 'Oct-2022'
branch 'Nov-2022' set up to track 'origin/Nov-2022'.
Switched to a new branch 'Nov-2022'
branch 'Dec-2022' set up to track 'origin/Dec-2022'.
Switched to a new branch 'Dec-2022'
Updating files: 100% (185/185), done.
branch 'Jan-2023' set up to track 'origin/Jan-2023'.
Switched to a new branch 'Jan-2023'
branch 'Feb-2023' set up to track 'origin/Feb-2023'.
Switched to a new branch 'Feb-2023'
Updating files: 100% (185/185), done.
branch 'Mar-2023' set up to track 'origin/Mar-2023'.
Switched to a new branch 'Mar-2023'
branch 'ICON-ModEx-v2.1-April-2023-replicate-01

# Load and plot data

In [ ]:
icon_modex_manual = pd.read_csv('ICON-ModEx_manual_iterations.csv')

# These iterations are the very first automated iterations
# presented at AGU2022. They were done in a "hindcast mode"
# to be able to compare to the manual iterations.
icon_modex_raw = pd.read_csv('ICON-ModEx_automated_iterations_raw.csv')

fig, ax = plt.subplots()
plt.errorbar(
    icon_modex_manual['ntrain'],
    icon_modex_manual[['hold-out','cv_avg']].max(axis=1),
    yerr=icon_modex_manual['cv_std'],
    fmt='k+',
    ecolor='lightgray',
    capsize=0)
#plt.errorbar(
#    icon_modex_raw['ntrain'],
#    icon_modex_raw['hold-out-avg'],
#    yerr=icon_modex_raw['hold-out-std'],
#    fmt='ko-',
#    ecolor='r',
#    capsize=5)
plt.errorbar(
    icon_modex_scale['ntrain'],
    icon_modex_scale['hold-out-avg'],
    icon_modex_scale['hold-out-std'],
    fmt='ko-',
    ecolor='r',
    capsize=5)
plt.errorbar(
    icon_modex_log10['ntrain'],
    icon_modex_log10['hold-out-avg'],
    icon_modex_log10['hold-out-std'],
    fmt='ks',
    ecolor='b',
    capsize=5)
ax.grid()
plt.ylabel('Avg ML Model Score of Training data set (R2)')
plt.xlabel('Total number of samples in input data set')
plt.legend(['Manual iterations','ModEx - live','ModEx - log10 hindcast'])
plt.savefig('ICON_ModEx_summary.pdf')

**Figure Caption:** Manual runs were done before AGU2022 as ICON-ModEx ramped up. The "live" iterations are the actual ICON-ModEx ML models that generated the results that were used for making decisions of which sites to sample. The hindcast iterations use a log10 filter because this reduces the bias of the ML models (see histogram analysis of Fig2). The big jump in the live iterations is due to the scaling of the respiration rate data with respect to the volume of the analysis container (need to double check this) (i.e. "Normalization") in June of 2023. The hindcast runs all take advantage of the scaling and the log10 transform to provide an apples-to-apples comparision of the progress of the ICON-ModEx iterations.

# Load and plot simplified version

In [ ]:
icon_modex_manual = pd.read_csv('ICON-ModEx_manual_iterations.csv')

# These iterations are the very first automated iterations
# presented at AGU2022. They were done in a "hindcast mode"
# to be able to compare to the manual iterations.
icon_modex_raw = pd.read_csv('ICON-ModEx_automated_iterations_raw.csv')

fig, ax = plt.subplots()
plt.errorbar(
    icon_modex_manual['ntrain'],
    icon_modex_manual[['hold-out','cv_avg']].max(axis=1),
    yerr=icon_modex_manual['cv_std'],
    fmt='k+',
    ecolor='lightgray',
    capsize=0)
plt.errorbar(
    icon_modex_log10['ntrain'],
    icon_modex_log10['hold-out-avg'],
    icon_modex_log10['hold-out-std'],
    fmt='ks',
    ecolor='b',
    capsize=5)
ax.grid()
plt.ylabel('Avg ML Model Score of Training data set (R2)')
plt.xlabel('Total number of samples in input data set')
plt.legend(['Manual pre-ModEx iterations','ModEx iterations'])
plt.savefig('ICON_ModEx_summary_simplified.pdf')

## Clean up

In [ ]:
#! rm -fv make_scatter_plot.ipynb